In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import functools
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
movies = pd.read_csv('movies_metadata.csv', usecols=['id', 'title'])
ratings = pd.read_csv('ratings_small.csv', usecols=['userId', 'movieId', 'rating'])

ratings.head(3)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0


In [ ]:
len(ratings['userId'].unique()), len(ratings['movieId'].unique()), ratings.shape

(671, 9066, (100004, 3))

671 пользователя, 9066 фильма

In [ ]:
invalid_rows = movies[~movies['id'].str.isnumeric()]
invalid_rows

,id,title
19730,1997-08-20,NaN
29503,2012-09-29,NaN
35587,2014-01-01,NaN


In [ ]:
movies = movies[movies['id'].str.isnumeric()]

In [ ]:
movies['id'] = movies['id'].astype(int)

Соединим 2 датасета: рейтинги пользователей и датасет о фильмах, по movieId


In [ ]:
info = pd.merge(ratings, movies, left_on='movieId', right_on='id').sort_values('userId')
info = info.drop('id', axis=1)

In [ ]:
info = info.drop_duplicates()

num_duplicates_after = info.duplicated().sum()
print(f'Количество дубликатов после удаления: {num_duplicates_after}')

Количество дубликатов после удаления: 0


In [ ]:
info.head()

,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
1,1,1405,1.0,Greed
2,1,2105,4.0,American Pie
3,1,2193,2.0,My Tutor
4,1,2294,2.0,Jay and Silent Bob Strike Back


Можем заметить, что фильмов у нас меньше чемв изначальном датасете, а movieId	соответсвуют тому датасету. Можем изменить их movieId	, основываясь теперь на уникальных значениях в этом датасете.

In [ ]:
uniq_m_id = info['movieId'].unique()
uniq_u_id = info['userId'].unique()

def calculate_m(x):
  ind = np.where(uniq_m_id == x)[0][0] + 1
  return ind

def calculate_u(x):
  ind = np.where(uniq_u_id == x)[0][0] + 1
  return ind

info['movieId'] = info['movieId'].apply(calculate_m)
info['userId'] = info['userId'].apply(calculate_u)

info.head()

,userId,movieId,rating,title
0,1,1,2.5,Rocky III
1,1,2,1.0,Greed
2,1,3,4.0,American Pie
3,1,4,2.0,My Tutor
4,1,5,2.0,Jay and Silent Bob Strike Back


Разобьем на трейн и тест наши данные.

In [ ]:
train_df, test_df = train_test_split(info, test_size=0.2)
train_df.shape, test_df.shape

((35991, 4), (8998, 4))

In [ ]:
train_df.head()

,userId,movieId,rating,title
4841,73,707,3.5,2010
44201,656,189,4.0,The Prize
44669,665,25,3.0,Wag the Dog
15475,243,212,3.0,Get Carter
19645,310,271,3.0,Interview with the Vampire


Есть 2 типа коллаборативной фильтрации: **Memory-Based Collaborative filtering** и **Model-Based Collaborative filtering**. Будем рассматривать оба метода, сравнивая их по метрике RMSE.

Для оценок качества моделей будем использовать:

$$ RMSE = \sqrt{\frac{1}{N}\sum (x_i - \hat{x}_i)^2} $$

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(pred, truth):
  predictions = np.nan_to_num(pred)[truth.nonzero()].flatten()
  truth = np.nan_to_num(truth)[truth.nonzero()].flatten()
  return sqrt(mean_squared_error(truth, predictions))

# Memory-Based Collaborative filtering
Есть 2 подхода:
1.  **user-user filtering** - рекомендации, основанные на пользователях, то есть мы ищем похожих пользователей.
2.  **item-item filtering** -  рекмоендации, основанные на объектах, ищем похожие фильмы.


Оба подхода основаны на мере схожести объектов, есть различные идеи для измерения этого. Один из них-это рассчет косинусного расстояние между векторами, в которых хранится описание пользователей и фильмов.

$$ s_{u}^{cos}= \frac{A\cdot  B}{\left \| A \right \| \left \| B \right \|} = \frac{\sum A_{i} B_{i}}{\sqrt{\sum A_{i}^2 \sum B_{i}^2}} $$

В обоих случаях нам необходимо создать user-item матрицу в каждой ячейке рейтинг от m-го пользователя n-му фильму.
Матрицы в подходах отличаются тем, что в user-item в строках находятся пользователи, в столбцах находятся фильмы.
В item-item в строках фильмы, в столбцах пользователи.

Для начала подготовим и сформируем матрицы:


In [ ]:
n_users = len(info['userId'].unique())
n_movies = len(info['movieId'].unique())

train_matrix, test_matrix = np.zeros((n_users , n_movies)), np.zeros((n_users , n_movies))

for line in train_df.itertuples():
  train_matrix[line[1] - 1, line[2] - 1] = line[3]

for line in test_df.itertuples():
  test_matrix[line[1] - 1, line[2] - 1] = line[3]

Посчитаем косинусное расстояние:

In [ ]:
from  sklearn.metrics.pairwise import pairwise_distances

user_sim = pairwise_distances(train_matrix, metric='cosine')
item_sim = pairwise_distances(train_matrix.T, metric='cosine')

В коде:

* user_sim[i][j] — косинусное расстояние между i-ой строкой и j-ой строкой;
* item_sim[i][j] — косинусное расстояние между i-ой и j-ой колонками.

Будем считать, что косинусное расстояние обозначает степень похожести: чем пользователи или фильмы более похожи друг на друга — тем меньше будет косинусное расстояние.

Есть несколько моделей для реализации этих подходов, в этой работе будет рассмотрено 3.

1. Первый вариант.

Это простой вариант реализации, основанный на среднем арифмитическом значении рейтинга u фильма i от n зрителей похожих на u-го пользователя. Похожих пользователей мы возьмем с меньшим значением косинусного расстояния.

$$ R = \frac{\sum r_k}{N} $$

*   U - пользователь, для которого предсказывают рейтинг
*   i - фильм, для которого предсказывают оценку пользователя U
*   $R$ - предсказанный рейтинг пользователя U за i-й фильм
*   $r_{k}$ - рейтинг k-го пользователя, похожего на U-го, за i-й фильм
*   $\sum r_{k}$ - сумма рейтингов пользователей, похожих на пользователя U
*   $N$ - количество похожих пользователей на пользователя U

In [ ]:
def first_model(top):
  predict = np.zeros((n_users, n_movies))
  for i in range(n_users):
    ind_sim = user_sim[i].argsort()[1: top + 1]
    ratings_sim = train_matrix[ind_sim]
    predict[i] = ratings_sim.sum(axis=0) / top
  return predict

def first_model_item(top):
  predict = np.zeros((n_movies, n_users))
  for i in range(n_movies):
    ind_sim = item_sim[i].argsort()[1: top + 1]
    ratings = train_matrix.T[ind_sim]
    predict[i] = ratings.sum(axis=0) / top
  return predict.T

In [ ]:
naive_predict = first_model(7)
naive_predict_item = first_model_item(7)
print(f'RMSE user-user: {rmse(naive_predict, test_matrix)}')
print(f'RMSE item-item: {rmse(naive_predict_item, test_matrix)}')

RMSE user-user: 2.8396588010390285
RMSE item-item: 2.9696794537594347


2.   Второй вариант.

Будем учитывать метрику похожести. То есть будем опираться на 2 матрицы: матрицу похожести и матрицу и оценок похожих пользователей.



$$ R = \frac{\sum (Sim_k \cdot r_k)}{\sum Sim_k} $$


*   $ R$ - предсказанный рейтинг пользователя U за i фильм.
*   $Sim_k$ - косинусные расстояния k похожих пользователей/фильмов.
*   $r_k$ - оценка пользователя k за i фильм.

В этой модели чем больше похожи пользователи/фильмы, тем сильнее оценка будет влиять на предсказание.

In [ ]:
def second_model(top):
  predict = np.zeros((n_users, n_movies))

  for i in range(n_users):
    ind_sim = user_sim[i].argsort()[1: top + 1]
    ratings = train_matrix[ind_sim]
    predict[i] = np.dot(user_sim[i][ind_sim], ratings) / user_sim[i][ind_sim].sum()
  return predict

def second_model_item(top):
  predict = np.zeros((n_movies, n_users))

  for i in range(n_movies):
    ind_sim = item_sim[i].argsort()[1: top + 1]
    ratings = train_matrix.T[ind_sim]
    predict[i] = np.dot(item_sim[i][ind_sim], ratings) / item_sim[i][ind_sim].sum()
  return predict.T

In [ ]:
second_predict = second_model(7)
second_predict_item = second_model_item(7)
print(f'RMSE user-user: {rmse(second_predict, test_matrix)}')
print(f'RMSE item-item: {rmse(second_predict_item, test_matrix)}')

RMSE user-user: 2.819340586569913
RMSE item-item: 2.9577243006586227


3.   Третий вариант.


Третий вариант основан на том, что учитывается среднее значение оценок u-го пользователя и среднее значения k похожих пользователей/фильмов и , как и во 2-м варианте, учитываются . Это учитывается потому, что одни пользователи всегде ставят оценки 4 и 5, а некоторые в диапазоне [1,3]. Это помогает более непредвзято прогнозировать.

$$ R = \overline{R} + \frac{\sum \left(Sim_k \cdot \left(r_k - \overline{r_k}\right)\right)}{\sum Sim} $$



*  $\overline{R}$ - средний рейтинг, который ставит u-пользователь  просмотренным фильмам.
*  $\overline{r_k}$ - средний рейтинг, который ставит k-пользователь просмотренным фильмам.



In [ ]:
def third_model(top):
  predict = np.zeros((n_users, n_movies))
  for i in range(n_users):
    ind_sim = user_sim[i].argsort()[1: top + 1]
    ratings = train_matrix[ind_sim]
    mean_rating = np.array([x for x in train_matrix[i] if x > 0]).mean()
    diff_ratings = ratings - train_matrix[ind_sim].mean()
    predict[i] = mean_rating  + np.dot(user_sim[i][ind_sim], diff_ratings) / user_sim[i][ind_sim ].sum()
  return predict

def third_model_item(top):
  predict = np.zeros((n_movies, n_users))
  for i in range(n_movies):
    ind_sim = item_sim[i].argsort()[1: top + 1]
    ratings = train_matrix.T[ind_sim]
    mean_rating = np.array([x for x in train_matrix.T[i] if x > 0]).mean()
    diff_ratings = ratings - train_matrix.T[ind_sim].mean()
    predict[i] = mean_rating + np.dot(item_sim[i][ind_sim], diff_ratings) / item_sim[i][ind_sim].sum()
  return predict.T

In [ ]:
third_predict = third_model(10)
third_predict_items = third_model_item(10)
print(f'RMSE user-user: {rmse(third_predict, test_matrix)}')
print(f'RMSE item-item: {rmse(third_predict_items, test_matrix)}')

RMSE user-user: 1.474293382418934
RMSE item-item: 1.4703611265892536


#  Model-Based Collaborative filtering

Oснован на разложении матриц. В данной работе используется метод разложения, который называется **сингулярное разложение**  (SVD, Singular Value Decomposition). Смысл этого разложения в том, что исходную матрицу $X$ мы разбиваем на произведение ортогональных матриц $U$ и $V^T$ и диагональной матрицы $Σ$.

$$ X = UΣV^T$$

Пусть $X$ — разреженная (в основном состоящая из нулей) матрица user-item.

В SVD лямбды в матрице $Σ$ (диагональные элементы) упорядочены по невозрастанию.

На основе SVD существует метод PCA, в котором из матрицы $Σ$ берут первые $d$ лямбд, а остальные полагаются равными 0. Это равносильно тому, что в $U$ и $V^T$ оставляются только первые $d$ столбцов. В результате получаем приближенную матрицу $\tilde{X}$, которая хорошо описывает исходную матрицу $X$ в меньшей размерности.

**SVD в системах рекомендаций**:

Мы приблизительно разложили исходную матрицу в произведение трех матриц. Давайте обозначим произведение первых двух сразу за одну матрицу:


$$ R = UV$$

Размерность матриц: $U$: $n×d$, $V$: $d×m$, где d-это количество скрытых интересов пользователей/ свойств фильмов.

Если мы хотим предсказать оценку $u$-го пользователя для $i$-го фильма, мы используем вектор $\mathbf{p_u}$ из матрицы $U$, который описывает эмбеддинги $u$-го пользователя, и вектор $\mathbf{q_i}$ из матрицы $V$, который описывает эмбеддинги фильма $i$. Их скалярное произведение даст предсказанный рейтинг:


$$ \hat{r}_{ui} = <{p_u},{q_i}>$$

Алгоритм достаточно простой, но дает хорошие результаты. Он не просто позволяет нам предсказывать оценки. С его помощью мы можем по истории пользователей выявлять скрытые признаки объектов и интересы пользователей.


Для того получить модель, которая будет предсказывать рейтинги пользователей, необходимо найти оптимальные параметры, при которых модель сможет лучше предсказывать рейтинги.

У нас есть матрица $R$, в которой уже есть проставленные рейтинги пользователями. Тогда будем решать задачу минимизации ошибки между проставленным рейтингом $u$-го пользователя для $i$-го фильма и предсказанным нашей моделью. Для того, чтобы модель не переобучалась под трейн, нужно добавить регулязационный член.

Итак,  хотим научиться как можно лучше приближать известные рейтинги, тогда получим такую функцию потерь:


$L =\sum_{(u, i) \in K}  (r_{ui} -  \hat{r}_{ui})^2 + \lambda(||\mathbf{p_u}||^2$ + $||\mathbf{q_i}||^2)$

$K$ — множество индексов $(u, i)$, для которых известны реальные рейтинги.

Оптимальные параметры можно найти с помощью стохастического градиентного спуска:


${p}_{uk} = {p}_{uk} + \text{step} \cdot \left( r_{ui} - \hat{r}_{ui} \right) \cdot {q}_{ki} - \lambda{p}_{uk}$

${q}_{ki} = {q}_{ki} + \text{step} \cdot \left( r_{ui} - \hat{r}_{ui} \right) \cdot {p}_{uk} - \lambda{q}_{ki}$

In [ ]:
from surprise import Dataset, Reader, accuracy, SVD
from surprise.model_selection import train_test_split


def SVD_model(df):
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(info[['userId', 'movieId', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.20)

    algo = SVD()
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions, verbose=True)

    return algo

In [ ]:
svd_model = SVD_model(info)

RMSE: 0.8874


Проблемы этого метода заключаются в том, что он, во-первых, работает очень медленно, когда очень много данных, и, во-вторых, находит локальные, а не глобальные минимумы. Вторая проблема не так страшна, т.к. в нашем случае значение функционала в локальных минимумах оказывается близким к глобальному оптимуму.

**Alternating Least Squares (ALS)**

Есть более быстрые и надёжные способы. Если мысленно заморозить параметры, соответствующие латентным факторам пользователей, задача оптимизации латентных представлений объектов сведётся к задаче наименьших квадратов, для которой мы знаем точное решение.
Итоговый процесс оптимизации функции потерь будет иметь следующий вид.
В цикле до сходимости:
*   Фиксируем матрицу $U$ (скрытые представления пользователей)
*   Решаем задачу L2-регуляризованной регрессии для каждого фильма и находим оптимальную матрицу $V$
*   Фиксируем матрицу $V$ (скрытые представления объектов)
*  Решаем задачу L2-регуляризованной регрессии для каждого пользователя и находим оптимальную матрицу $U$

Решение, получаемое путём попеременного вычисления точных аналитических решений, обычно точнее тех, что получаются с помощью наивного градиентного спуска. Более того, данное решение имеет эффективную реализацию, позволяющую использовать преимущества параллельных вычислений.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

spark = SparkSession.builder.appName("ExplicitALS").getOrCreate()

spark_df = spark.createDataFrame(info[['userId', 'movieId', 'rating']])


als = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating", implicitPrefs=False)

train_als, test_als = spark_df.randomSplit([0.8, 0.2], seed=42)

als_model = als.fit(train_als)
predictions = als_model.transform(test_als)

predictions_array = np.array(predictions.select("prediction").rdd.flatMap(lambda x: x).collect())
truth_array = np.array(test_als.select("rating").rdd.flatMap(lambda x: x).collect())

metric = rmse(predictions_array, truth_array)

print(f"RMSE: {metric}")

RMSE: 1.3693244653563605


Теперь можно сравнить метрику RMSE у трех моделей:

1.   **Memory-based (user-user и item-item): 1.47**

Прост в понимании. Не требует предварительной обработки данных или обучение модели. Чувствителен к разреженности данных: если недостаточно взаимодействия пользователя и фильма, то вычисление сходства может стать недостоверным. Также при увеличении объемов данных, возрастает время вычисления данных.

2.   **SVD: 0.89**

Наилучший показатель RMSE. SVD хорошо подходит для факторизации матрицы, извлекая скрытые связи между пользователями и фильмами, что полезно в разреженных данных. Также уменьшает размерность данных, что позволяет ускорить вычисления. Однако, вычисление стохастического градиентного спуска требует большего времени-> увеличивает скорость работы. Также при добавлении новых пользователей/фильмов, придется пересчитывать все заново-долго и дорого.


3.   **ALS: 1.37**

Может работать с разреженными данными, подобно SVD, но также оптимизирован для обработки больших объемов данных.Подходит для явных и неявных взаимодействий.

Основной недостаток методов коллаборативной рекомендации то, что они используют лишь информацию о взаимодействии пользователей и объектов, но не о них самих. Для этого используют content-based,которые используют атрибуты объектов и пользователей.


Выведем топ-10 рекомендованных фильмов для пользователя 1, используя метод с наименьшим значением метрики RMSE.

In [ ]:
def generate_recommendations_svd(user_id, algo, df):
    """Выводим топ10 фильмов для пользователя"""
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data.build_full_trainset()
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)

    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Top 10 рекомендаций для {user_id}:")
    for num, pred in enumerate(top_10_recommendations):
        movie_title = df.loc[df['movieId'] == pred.iid, 'title'].values[0]
        print(f"{num}. {movie_title}, Estimated Rating: {pred.est:.2f}")

In [ ]:
 generate_recommendations_svd(1, svd_model, info)

Top 10 рекомендаций для 1:
Movie: 5 Card Stud, Estimated Rating: 4.19
Movie: Sleepless in Seattle, Estimated Rating: 4.16
Movie: Galaxy Quest, Estimated Rating: 4.15
Movie: The Million Dollar Hotel, Estimated Rating: 4.12
Movie: Madagascar, Estimated Rating: 4.11
Movie: The Thomas Crown Affair, Estimated Rating: 4.10
Movie: Pandora's Box, Estimated Rating: 3.98
Movie: While You Were Sleeping, Estimated Rating: 3.98
Movie: Once Were Warriors, Estimated Rating: 3.97
Movie: Under the Sand, Estimated Rating: 3.95
